In [6]:
import polars as pl

In [7]:
DATA = '../data'
f = f'{DATA}/legislators-current.csv'

In [9]:
dtypes = {
    field: pl.Categorical
    for field in ['first_name', 'gender', 'type', 'state', 'party']}
df = (
    pl
    .read_csv(f, dtype=dtypes)
    .with_column(pl.col('birthday').str.strptime(pl.Date, strict=False))
)
df.head()

/var/folders/41/jhvt_zmd3nz3x5jsqblr221r0000gn/T/ipykernel_27857/1296176373.py:5: DeprecationWarning: `dtype` is deprecated as an argument to `read_csv`; use `dtypes` instead.
  pl


last_name,first_name,middle_name,suffix,nickname,full_name,birthday,gender,type,state,district,senate_class,party,url,address,phone,contact_form,rss_url,twitter,twitter_id,facebook,youtube,youtube_id,mastodon,bioguide_id,thomas_id,opensecrets_id,lis_id,fec_ids,cspan_id,govtrack_id,votesmart_id,ballotpedia_id,washington_post_id,icpsr_id,wikipedia_id
str,cat,str,str,str,str,date,cat,cat,cat,i64,i64,cat,str,str,str,str,str,str,i64,str,str,str,str,str,i64,str,str,str,i64,i64,i64,str,str,i64,str
"""Brown""","""Sherrod""",null,null,null,"""Sherrod Brown""",1952-11-09,"""M""","""sen""","""OH""",null,1,"""Democrat""","""https://www.br...","""503 Hart Senat...","""202-224-2315""","""https://www.br...","""http://www.bro...","""SenSherrodBrow...",43910797,"""SenatorSherrod...","""SherrodBrownOh...","""UCgy8jfERh-t_i...",null,"""B000944""",136,"""N00003535""","""S307""","""H2OH13033,S6OH...",5051,400050,27018,"""Sherrod Brown""",null,29389,"""Sherrod Brown"""
"""Cantwell""","""Maria""",null,null,null,"""Maria Cantwell...",1958-10-13,"""F""","""sen""","""WA""",null,1,"""Democrat""","""https://www.ca...","""511 Hart Senat...","""202-224-3441""","""https://www.ca...","""http://www.can...","""SenatorCantwel...",117501995,"""senatorcantwel...","""SenatorCantwel...","""UCN52UDqKgvHRk...",null,"""C000127""",172,"""N00007836""","""S275""","""S8WA00194,H2WA...",26137,300018,27122,"""Maria Cantwell...",null,39310,"""Maria Cantwell..."
"""Cardin""","""Benjamin""","""L.""",null,null,"""Benjamin L. Ca...",1943-10-05,"""M""","""sen""","""MD""",null,1,"""Democrat""","""https://www.ca...","""509 Hart Senat...","""202-224-4524""","""https://www.ca...","""http://www.car...","""SenatorCardin""",109071031,"""senatorbencard...","""senatorcardin""","""UCiQaJnMzlfzzG...",null,"""C000141""",174,"""N00001955""","""S308""","""H6MD03177,S6MD...",4004,400064,26888,"""Ben Cardin""",null,15408,"""Ben Cardin"""
"""Carper""","""Thomas""","""Richard""",null,null,"""Thomas R. Carp...",1947-01-23,"""M""","""sen""","""DE""",null,1,"""Democrat""","""https://www.ca...","""513 Hart Senat...","""202-224-2441""","""https://www.ca...","""http://www.car...","""SenatorCarper""",249787913,"""tomcarper""","""senatorcarper""","""UCgLnvbKwu4B3n...",null,"""C000174""",179,"""N00012508""","""S277""","""S8DE00079""",663,300019,22421,"""Tom Carper""",null,15015,"""Tom Carper"""
"""Casey""","""Robert""","""P.""","""Jr.""","""Bob""","""Robert P. Case...",1960-04-13,"""M""","""sen""","""PA""",null,1,"""Democrat""","""https://www.ca...","""393 Russell Se...","""202-224-6324""","""https://www.ca...","""http://www.cas...","""SenBobCasey""",171598736,"""SenatorBobCase...","""SenatorBobCase...","""UCtVssXhx-KuZa...",null,"""C001070""",1828,"""N00027503""","""S309""","""S6PA00217""",47036,412246,2541,"""Bob Casey, Jr....",null,40703,"""Bob Casey Jr."""


In [10]:
q = (
    df
    .lazy()
    .groupby('first_name')
    .agg([
        pl.count(),
        pl.col('gender').list(),
        pl.first('last_name')])
    .sort('count', reverse=True)
    .limit(10))
q.collect()

first_name,count,gender,last_name
cat,u32,list[cat],str
"""John""",21,"[""M"", ""M"", ... ""M""]","""Barrasso"""
"""Mike""",15,"[""M"", ""M"", ... ""M""]","""Kelly"""
"""Michael""",10,"[""M"", ""M"", ... ""M""]","""Bennet"""
"""Robert""",9,"[""M"", ""M"", ... ""M""]","""Casey"""
"""Mark""",8,"[""M"", ""M"", ... ""M""]","""Warner"""
"""David""",8,"[""M"", ""M"", ... ""M""]","""Cicilline"""
"""James""",7,"[""M"", ""M"", ... ""M""]","""Risch"""
"""Bill""",7,"[""M"", ""M"", ... ""M""]","""Cassidy"""
"""Kevin""",5,"[""M"", ""M"", ... ""M""]","""McCarthy"""
